In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

ltbxd_raw = pd.read_csv(r'../data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\watched.csv')

In [21]:
ltbxd_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            941 non-null    object
 1   Name            941 non-null    object
 2   Year            941 non-null    int64 
 3   Letterboxd URI  941 non-null    object
dtypes: int64(1), object(3)
memory usage: 29.5+ KB


## Datascraper - Adicionando os dados

In [22]:
ltbxd_raw['Director'] = np.nan

ltbxd_raw['Genres'] = np.nan

# Assuming ltbxd_raw is your DataFrame
for index, row in ltbxd_raw.iterrows():
    url = row['Letterboxd URI']
    
    # Add user-agent headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    # Sending a GET request to the URL
    response = requests.get(url, headers=headers)
    
    # Checking if the request was successful (status code 200)
    if response.status_code == 200:
        # Parsing the HTML content of the response
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Now you can work with the parsed HTML content
        # For example, you can extract specific information from the webpage
        
        # For demonstration, let's print the title of the webpage
        print(f"Title of webpage {index}: {soup.title.string}")
        # Parse the HTML

        # Find the span with class 'directorlist'
        director_span = soup.find('span', class_='directorlist')

        # Extract the director's name from the span
        director_name = director_span.text.strip()
        
        ltbxd_raw.at[index, 'Director'] = director_name
        
        # Extract genres from the specific section
        genre_section = soup.find('div', id='tab-genres')
        if genre_section:
            genre_tags = genre_section.find_all('a', class_='text-slug')
            genres = [tag.text for tag in genre_tags if '/films/genre/' in tag['href']]
        else:
            genres = ["Genres not found"]
        
        # Extract themes from the specific section
        if genre_section:
            theme_tags = genre_section.find_all('a', class_='text-slug')
            themes = [tag.text for tag in theme_tags if '/films/theme/' in tag['href'] or '/films/mini-theme/' in tag['href']]
        else:
            themes = ["Themes not found"]
        
        # Update the DataFrame with the genres and themes
        ltbxd_raw.at[index, 'Genres'] = '; '.join(genres)
        ltbxd_raw.at[index, 'Themes'] = '; '.join(themes)
    else:
        print(f"Failed to fetch data from URL {url} with status code {response.status_code}")

Title of webpage 0: ‎Barbie (2023) directed by Greta Gerwig • Reviews, film + cast • Letterboxd


C:\Users\guibe\AppData\Local\Temp\ipykernel_11088\893918509.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Greta Gerwig' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ltbxd_raw.at[index, 'Director'] = director_name
C:\Users\guibe\AppData\Local\Temp\ipykernel_11088\893918509.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Comedy; Adventure' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ltbxd_raw.at[index, 'Genres'] = '; '.join(genres)


In [24]:
#adding ratings
ratings_raw =  pd.read_csv(r'../data\raw\letterboxd-gbmonteiro-2024-05-14-22-51-utc\ratings.csv')

ltbxd_wtchd = ltbxd_raw.merge(ratings_raw[['Name','Rating']],how='left',on='Name')

## PG connection - Adicionando dados no postgres